論文  
https://arxiv.org/abs/2112.04685<br>
<br>  
GitHub  
https://github.com/haoheliu/2021-ISMIR-MSS-Challenge-CWS-PResUNet<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/CWS_PResUNet_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 環境セットアップ

## GPU確認

In [ ]:
!nvidia-smi

## GitHubからコード取得

In [ ]:
%cd /content

!git clone https://github.com/haoheliu/2021-ISMIR-MSS-Challenge-CWS-PResUNet.git

## ライブラリのインストール

In [ ]:
%cd /content/2021-ISMIR-MSS-Challenge-CWS-PResUNet

!pip install --upgrade gdown

! pip3 install -r requirements.txt
# ModuleNotFoundError: No module named 'torchtext.legacy' 回避のため torchtextはバージョン固定
! pip3 install torchtext==0.8.1 demucs==3.0.1

# Vocal抽出(テスト音源)

In [ ]:
%cd /content/2021-ISMIR-MSS-Challenge-CWS-PResUNet

!python3 main.py -i example/test -o example/results -s vocals other --cuda 

# 抽出結果再生

In [ ]:
import IPython.display as ipd
ipd.Audio("example/test/xuemaojiao.wav", rate=44100)

In [ ]:
ipd.Audio("example/results/xuemaojiao/vocals.wav", rate=44100)

# Vocal抽出(Youtube)

## 動画のセットアップ

In [ ]:
!pip install yt-dlp moviepy 

In [ ]:
from yt_dlp import YoutubeDL

from moviepy.video.fx.resize import resize
from moviepy.editor import VideoFileClip, AudioFileClip, ImageSequenceClip, CompositeAudioClip
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

In [ ]:
video_url = 'https://www.youtube.com/watch?v=Qd01-6xVSHk' #@param {type:"string"}

#@markdown 動画の切り抜き範囲(秒)を指定してください。\
#@markdown 30秒以上の場合OOM発生の可能性が高いため注意
start_sec =  57#@param {type:"integer"}
end_sec =  69#@param {type:"integer"}

(start_pt, end_pt) = (start_sec, end_sec)

## 動画取得

In [ ]:
!mkdir -p /content/2021-ISMIR-MSS-Challenge-CWS-PResUNet/videos/results


download_resolution = 360
full_video_path = '/content/2021-ISMIR-MSS-Challenge-CWS-PResUNet/videos/full_video.mp4'
input_clip_path = '/content/2021-ISMIR-MSS-Challenge-CWS-PResUNet/videos/input_clip.mp4'
input_audio_path = '/content/2021-ISMIR-MSS-Challenge-CWS-PResUNet/videos/input_clip.wav'

# 動画ダウンロード
ydl_opts = {'format': f'best[height<={download_resolution}]', 'overwrites': True, 'outtmpl': full_video_path}
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

## 指定区間切り抜き

In [ ]:
# extract the relevant subclip:
with VideoFileClip(full_video_path) as video:
    subclip = video.subclip(start_pt, end_pt)
    subclip.write_videofile(input_clip_path)

## 切り抜き箇所(0~5秒)再生

In [ ]:
clip = VideoFileClip(input_clip_path)

if 5 < (end_pt-start_pt):
  clip = clip.subclip(0, 5)
  
clip = resize(clip, height=420)
clip.ipython_display()

## audioファイル取得

In [ ]:
clip = VideoFileClip(input_clip_path)
clip.audio.write_audiofile(input_audio_path, codec='pcm_s16le')

ipd.Audio(input_audio_path, rate=44100)

## Vocal抽出

In [ ]:
!python3 main.py -i {input_audio_path} -o /content/2021-ISMIR-MSS-Challenge-CWS-PResUNet/videos/results -s vocals other --cuda 

### Vocal

In [ ]:
ipd.Audio('/content/2021-ISMIR-MSS-Challenge-CWS-PResUNet/videos/results/input_clip/vocals.wav', rate=44100)

### Vocal以外

In [ ]:
ipd.Audio('/content/2021-ISMIR-MSS-Challenge-CWS-PResUNet/videos/results/input_clip/other.wav', rate=44100)

## Vocalのみの動画を作成

In [ ]:
videoclip = VideoFileClip(input_clip_path)
audioclip = AudioFileClip("/content/2021-ISMIR-MSS-Challenge-CWS-PResUNet/videos/results/input_clip/vocals.wav")

new_audioclip = CompositeAudioClip([audioclip])
videoclip.audio = new_audioclip
videoclip.write_videofile("/content/2021-ISMIR-MSS-Challenge-CWS-PResUNet/videos/results/input_clip/vocals.mp4")

In [ ]:
clip = VideoFileClip('/content/2021-ISMIR-MSS-Challenge-CWS-PResUNet/videos/results/input_clip/vocals.mp4')
  
clip = resize(clip, height=420)
clip.ipython_display()